In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: Tesla T4


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "/content/drive/MyDrive/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


In [ ]:
# Chia dữ liệu thành tập train và tập test
train_dataset, test_dataset = random_split(Image_dataset, [0.7, 0.3])

batch_size = 32
# Tạo DataLoader để nạp dữ liệu theo batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(Image_dataset)

1346

In [ ]:
from torchvision.models import efficientnet_b0
import torch.nn as nn

class VisionModel(nn.Module):
    def __init__(self, num_classes=16):
        super(VisionModel, self).__init__()

        self.efficientnet_b0 = efficientnet_b0(pretrained=False).to(device)

        n_inputs = self.efficientnet_b0.classifier[1].in_features
        self.efficientnet_b0.classifier[1] = nn.Sequential(
            nn.Linear(n_inputs, num_classes, bias=True),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.efficientnet_b0(x)
        return x

num_classes = len(Image_dataset.classes)
vision_model = VisionModel(num_classes).to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
print(vision_model)

VisionModel(
  (efficientnet_b0): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        # Chuyển dữ liệu và mô hình lên GPU
        X, y = X.to(device), y.to(device)
        model.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X, y) in data_loader:
            # Chuyển dữ liệu lên GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(vision_model.parameters(), lr=0.0001)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()  # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc


In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    vision_model.train()

    train_step(data_loader=train_loader,
        model=vision_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )

    vision_model.eval()
    test_step(data_loader=test_loader,
        model=vision_model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

Epoch: 0
---------
Train loss: 2.72064 | Train accuracy: 15.25%
Test loss: 2.72022 | Test accuracy: 13.70%

Epoch: 1
---------
Train loss: 2.57494 | Train accuracy: 16.62%
Test loss: 2.68603 | Test accuracy: 14.03%

Epoch: 2
---------
Train loss: 2.46577 | Train accuracy: 21.12%
Test loss: 2.59280 | Test accuracy: 21.10%

Epoch: 3
---------
Train loss: 2.36929 | Train accuracy: 25.60%
Test loss: 2.84815 | Test accuracy: 25.73%

Epoch: 4
---------
Train loss: 2.25036 | Train accuracy: 27.23%
Test loss: 2.78632 | Test accuracy: 27.26%

Epoch: 5
---------
Train loss: 2.10790 | Train accuracy: 33.01%
Test loss: 2.81633 | Test accuracy: 30.07%

Epoch: 6
---------
Train loss: 1.98455 | Train accuracy: 36.49%
Test loss: 3.02947 | Test accuracy: 32.00%

Epoch: 7
---------
Train loss: 1.83255 | Train accuracy: 39.62%
Test loss: 2.65563 | Test accuracy: 30.71%

Epoch: 8
---------
Train loss: 1.65752 | Train accuracy: 50.15%
Test loss: 2.25705 | Test accuracy: 39.85%

Epoch: 9
---------
Train los